In [7]:
from sklearn.feature_extraction import DictVectorizer
from allennlp.data.tokenizers import Tokenizer, WordTokenizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import csv
stemmer = nltk.stem.PorterStemmer()
nltk.download('stopwords')
nltk.download('wordnet')
stopWords = set(stopwords.words('english'))
stopWords.add("'s")
lemmatizer = WordNetLemmatizer()
from allennlp.data.tokenizers import Tokenizer, WordTokenizer
wordTokenizer = WordTokenizer() 

[nltk_data] Downloading package stopwords to /home/shawn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/shawn/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
train = []
devset = []
with open('/home/shawn/workspace/research/final_codelab/data/final/classification_model/jun/train.tsv','r') as f:
        csv_reader = csv.reader(f, delimiter='\t')
        for row in csv_reader:
            train.append(row)
with open('/home/shawn/workspace/research/final_codelab/shawn/input/classification/devset/test.tsv','r') as f:
        csv_reader = csv.reader(f, delimiter='\t')
        for row in csv_reader:
            row.append('NOT_DEFINED')
            devset.append(row)
devset = devset[1:]
all = train + devset
print(len(train))
print(len(devset))
print(len(all))
assert len(all) == len(train) + len(devset)

145449
5001
150450


In [17]:
[1,2,3,4,5,6][4:]

[5, 6]

In [12]:
class BOWReprsentation:
    def __init__(self, dataset):
        self.wordTokenizer = WordTokenizer() 
        self.dataset = dataset
        
    def preprocess(self, text: str) -> list:
        text_list = self.wordTokenizer.tokenize(text)
        new_list = []
        for e in text_list:
            if e not in stopWords:
                new_list.append(e)
        return new_list

    # bag of words(BOW) feature representation
    def get_BOW(self, text: str) -> dict:
        BOW = {}
        pre_text = self.preprocess(text)
        for word in pre_text:
            word = str(word)
            BOW[word] = BOW.get(word,0) + 1
        return BOW
    
    def run(self):
        feature_matrix_claim = []
        feature_matrix_content = []
        classifications = []
        for row in self.dataset:
            claim = row[0]
            claim_bow = self.get_BOW(claim)
            content = row[1]
            content_bow = self.get_BOW(content)
            feature_matrix_claim.append(claim_bow)
            feature_matrix_content.append(content_bow)
            classifications.append(row[2])
        print(len(feature_matrix_claim))
        print(feature_matrix_claim[0])
        print(feature_matrix_content[0])
        vectorizer = DictVectorizer()
        dataset_claim = vectorizer.fit_transform(feature_matrix_claim)
        dataset_content = vectorizer.fit_transform(feature_matrix_content)
        return dataset_claim, dataset_content, classifications, feature_matrix_claim, feature_matrix_content

In [13]:
all_claim, all_content, classifications, all_matrix_claim, all_matrix_content = BOWReprsentation(all).run()

150450
{'Amazon': 1, 'is': 1, 'a': 1, 'more': 1, 'valuable': 1, 'retailer': 1, 'than': 1, 'Walmart': 1, '.': 1}
{'Amazon.com': 1, ',': 3, 'In': 1, '2015': 1, 'Amazon': 1, 'surpassed': 1, 'Walmart': 1, 'as': 1, 'the': 2, 'most': 1, 'valuable': 1, 'retailer': 1, 'in': 1, 'United': 1, 'States': 1, 'by': 1, 'market': 1, 'capitalization': 1, '.': 1}


In [24]:
print(all_claim.shape)
print(all_content.shape)
print(merged_feature.shape)

(150450, 26079)
(150450, 55099)
(1, 2)


In [31]:
merged_feature

array([[<26079x150450 sparse matrix of type '<class 'numpy.float64'>'
	with 1392911 stored elements in Compressed Sparse Column format>,
        <55099x150450 sparse matrix of type '<class 'numpy.float64'>'
	with 5759920 stored elements in Compressed Sparse Column format>]],
      dtype=object)

In [47]:
import scipy as sp
from scipy.sparse import hstack
## conctate the claim and content
# working
a = all_claim
a_train = a[0:len(train)]
a_devset = a[len(train):]
print(type(all_claim))
print(all_claim.shape)
b = all_content
b_train = b[0:len(train)]
b_devset = b[len(train):]
print(type(all_content))
print(all_content.shape)

X_train = hstack((a_train,b_train))  # NOT
X_devset = hstack((a_devset,b_devset))  # NOT
# trainingset
print(X_train.shape)
print(type(X_train))
# devset
print(X_devset.shape)
print(type(X_devset))

y_train = classifications[0:len(train)]
y_devset = classifications[len(train):]
assert len(y_train) == len(train)
assert len(y_devset) == len(devset)

<class 'scipy.sparse.csr.csr_matrix'>
(150450, 26079)
<class 'scipy.sparse.csr.csr_matrix'>
(150450, 55099)
(145449, 81178)
<class 'scipy.sparse.coo.coo_matrix'>
(5001, 81178)
<class 'scipy.sparse.coo.coo_matrix'>


In [54]:
devset_label

{'91198': {'claim': 'Colin Kaepernick became a starting quarterback during the 49ers 63rd season in the National Football League.',
  'label': 'NOT ENOUGH INFO',
  'evidence': []},
 '194462': {'claim': 'Tilda Swinton is a vegan.',
  'label': 'NOT ENOUGH INFO',
  'evidence': []},
 '137334': {'claim': 'Fox 2000 Pictures released the film Soul Food.',
  'label': 'SUPPORTS',
  'evidence': [['Soul_Food_-LRB-film-RRB-', 0]]},
 '166626': {'claim': 'Anne Rice was born in New Jersey.',
  'label': 'NOT ENOUGH INFO',
  'evidence': []},
 '111897': {'claim': 'Telemundo is a English-language television network.',
  'label': 'REFUTES',
  'evidence': [['Telemundo', 5],
   ['Telemundo', 4],
   ['Telemundo', 1],
   ['Telemundo', 0],
   ['Hispanic_and_Latino_Americans', 0]]},
 '89891': {'claim': "Damon Albarn's debut album was released in 2011.",
  'label': 'REFUTES',
  'evidence': [['Damon_Albarn', 17]]},
 '181634': {'claim': 'There is a capital called Mogadishu.',
  'label': 'SUPPORTS',
  'evidence': [

In [58]:
with open('/home/shawn/workspace/research/final_codelab/submission_data/original/devset.json') as fp:
    devset_label = json.load(fp)
y_devset = []
for key in devset_label.keys():
    if devset[key]['label']=='SUPPORTS':
        y_devset.append('1')
    elif devset_label[key]['label']=='REFUTES':
        y_devset.append('0')
    else:
        y_devset.append('2')
print(len(y_devset))

5001


In [49]:
# training
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection 
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

/home/shawn/anaconda3/envs/python36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [59]:
# prediction
import sklearn
from sklearn.metrics import accuracy_score
# evaluation
predictions_proba = clf.predict_proba(X_devset)
y_predictions = clf.predict(X_devset)
sklearn.metrics.accuracy_score(y_devset, y_predictions)

0.4433113377324535